---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
degrees = nx.degree(P1_Graphs[1])

In [3]:
import pandas as pd
#print(pd.Series(np.arange(len(P1_Graphs))).apply(lambda x:nx.average_shortest_path_length(P1_Graphs[x])))
#print(pd.Series(np.arange(len(P1_Graphs))).apply(lambda x:nx.average_clustering(P1_Graphs[x])))

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [4]:
def graph_identification():
    
    return (['PA','SW_L','SW_L','PA','SW_H'])

---

## Part 2 - Company Emails

For the second part of this assignment you will be working with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0

Length: 252, dtype: float64

In [6]:
# import packages
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import scale

def salary_predictions():

    # create dataframe containing potential pertinent features
    df = pd.DataFrame(index = G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G,'Department')) # not relevant after grid search
    df['Closeness_centrality'] = pd.Series(nx.closeness_centrality(G,normalized=True))
    df['Degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['Betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G,normalized=True))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    
    # scale x
   #df.iloc[:,:-1] = df.iloc[:,:-1].apply(lambda x:scale(x))
    
    # split data
    ids = df.iloc[:,-1][df.iloc[:,-1].isnull() == 1].index # ids for missing values
    X_train = df.iloc[~df.index.isin(ids),:-1] # train
    X_test = df.iloc[ids,:-1]
    y_train = df.iloc[~df.index.isin(ids),-1]
                            
    # fit rf with best parameters from Grid Search, n = 30, max_depth = 5
    rf = RandomForestClassifier(n_estimators=30,max_depth=5,random_state=0).fit(X_train,y_train)
    pred_prob = pd.Series(rf.predict_proba(X_test)[:,1])
    pred_prob.index = ids

    return pred_prob

In [7]:
# To find optimal parameters
def grid_search_params(df):
    df_train = df.dropna()

    # split data
    X_train,X_test,y_train,y_test = train_test_split(df_train.iloc[:,:-1],df_train.iloc[:,-1],random_state= 0)

    # GridSearch for parameters
    n_estimators = [10,20,30,40,50]
    max_depth = [int(x) for x in np.arange(5,8)]
    clf = RandomForestClassifier(random_state = 0) # classifier
    param_grid = {'n_estimators':n_estimators,'max_depth':max_depth}
    cv = 3
    
    rmGrid = GridSearchCV(clf,cv=cv,param_grid= param_grid,scoring='roc_auc') # implement gridsearch
    rmGrid.fit(X_train,y_train) 
    y_decision_fn_acc = rmGrid.predict_proba(X_test) # get pred probability
    
    return (roc_auc_score(y_test,y_decision_fn_acc[:,1]),rmGrid.best_params_) # 5,30

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [8]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [12]:
from sklearn.linear_model import LogisticRegression as lg

def new_connections_predictions():
    # map link prediction metrics to future_connections
    future_connections['CommonNeighbors'] = future_connections.index.map(lambda x: len(list(nx.common_neighbors(G, x[0], x[1]))))
    future_connections['PreferentialAttachment'] = [i[2] for i in nx.preferential_attachment(G,future_connections.index)]
    future_connections['Adamic_Adar'] =  [i[2] for i in nx.adamic_adar_index(G,future_connections.index)]
    future_connections['ResourceAllocation'] =  [i[2] for i in nx.resource_allocation_index(G,future_connections.index)]
    
    # scale x
    future_connections_scaled = pd.DataFrame()
    future_connections_scaled = future_connections.iloc[:,1:].apply(lambda x:scale(x))
    future_connections.iloc[:,1:] = future_connections_scaled
    
    # split data
    ids = future_connections[future_connections.iloc[:0].isnull() == 1].index # ids for missing values
    X_train = future_connections.iloc[~future_connections.index.isin(ids),1:] # train
    X_test = future_connections.iloc[future_connections.index.isin(ids),1:]
    y_train = future_connections.iloc[~future_connections.index.isin(ids),0]
                            
    # fit rf with best parameters from Grid Search, n = 30, max_depth = 5
    rf = lg(C = .001,random_state=0).fit(X_train,y_train)
    pred_prob = pd.Series(rf.predict_proba(X_test)[:,1])
    pred_prob.index = ids
    return pred_prob

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


TypeError: '>=' not supported between instances of 'tuple' and 'int'